### 1. Imports

In [5]:
import requests
import json
import pandas as pd

### 2. Get Data

In [6]:
uri = 'https://api.football-data.org/v4/competitions/BL1/matches?season=2020'
headers = {'X-Auth-Token': '941e5424e9374e28817b075859d2867c'}

In [7]:
response = requests.get(uri, headers=headers)

In [8]:
df = pd.DataFrame.from_dict(response.json().items())

In [9]:
df.rename(columns={0: 'item', 1: 'entry'}, inplace=True)

In [10]:
df

,item,entry
0,message,Your API token is invalid.
1,errorCode,400


### 3. Extract relevant data - OLD

In [85]:
home_team = df.iloc[3, 1][0]['homeTeam']['name']
away_team = df.iloc[3, 1][0]['awayTeam']['name']
score_home = df.iloc[3, 1][0]['score']['fullTime']['home']
score_away = df.iloc[3, 1][0]['score']['fullTime']['away']
winner = df.iloc[3, 1][0]['season']['winner']['name']

In [99]:
df.iloc[3, 1][0]

{'area': {'id': 2088,
  'name': 'Germany',
  'code': 'DEU',
  'flag': 'https://crests.football-data.org/759.svg'},
 'competition': {'id': 2002,
  'name': 'Bundesliga',
  'code': 'BL1',
  'type': 'LEAGUE',
  'emblem': 'https://crests.football-data.org/BL1.png'},
 'season': {'id': 599,
  'startDate': '2020-09-18',
  'endDate': '2021-05-22',
  'currentMatchday': 34,
  'winner': {'id': 5,
   'name': 'FC Bayern München',
   'shortName': 'Bayern',
   'tla': 'FCB',
   'crest': 'https://crests.football-data.org/5.svg',
   'address': 'Säbenerstr. 51 München 81547',
   'website': 'http://www.fcbayern.de',
   'founded': 1900,
   'clubColors': 'Red / White / Blue',
   'venue': 'Allianz Arena',
   'lastUpdated': '2021-12-16T18:52:55Z'}},
 'id': 303007,
 'utcDate': '2020-09-18T18:30:00Z',
 'status': 'FINISHED',
 'matchday': 1,
 'stage': 'REGULAR_SEASON',
 'group': 'Regular Season',
 'lastUpdated': '2020-10-03T07:30:16Z',
 'homeTeam': {'id': 5,
  'name': 'FC Bayern München',
  'shortName': 'Bayern',


### 3. Get user input 

In [11]:
# Create dict of available teams
team_dict = {'bayern': 40,
             'dortmund': 7}

In [13]:
home_team = input("What's the name of the home team?")
home_team = home_team.lower()

What's the name of the home team? bayern


In [51]:
away_team = input("What's the name of the away team?")
away_team = away_team.lower()

What's the name of the away team? dortmund


In [52]:
# Check if team exists
def check_team(home_team, away_team, team_dict):
    if home_team in team_dict and away_team in team_dict:
        home = team_dict[home_team]
        away = team_dict[away_team]
        return home, away
    else:
        return None, None

In [53]:
home, away = check_team(home_team, away_team, team_dict)

### 4. API call and get the data

In [ ]:
team_dict = {'BAYERN': 40,
             'DORTMUND': 7}

In [ ]:
team_dict['DORTMUND']

7

In [132]:
home_team = input()
print(home_team)

 7


7


In [133]:
home_team

'7'

In [131]:
url = 'https://api.openligadb.de/getmatchdata/40/7'
response = requests.get(url)

In [130]:
response.json()[0]

{'matchID': 63864,
 'matchDateTime': '2022-08-05T20:30:00',
 'timeZoneID': 'W. Europe Standard Time',
 'leagueId': 4562,
 'leagueName': '1. Fußball-Bundesliga 2022/2023',
 'leagueSeason': 2022,
 'leagueShortcut': 'bl1',
 'matchDateTimeUTC': '2022-08-05T18:30:00Z',
 'group': {'groupName': '1. Spieltag', 'groupOrderID': 1, 'groupID': 39881},
 'team1': {'teamId': 91,
  'teamName': 'Eintracht Frankfurt',
  'shortName': 'Frankfurt',
  'teamIconUrl': 'https://i.imgur.com/X8NFkOb.png',
  'teamGroupName': None},
 'team2': {'teamId': 40,
  'teamName': 'FC Bayern München',
  'shortName': 'Bayern',
  'teamIconUrl': 'https://i.imgur.com/jJEsJrj.png',
  'teamGroupName': None},
 'lastUpdateDateTime': '2022-08-14T09:42:12.96',
 'matchIsFinished': True,
 'matchResults': [{'resultID': 98425,
   'resultName': 'Endergebnis',
   'pointsTeam1': 1,
   'pointsTeam2': 6,
   'resultOrderID': 1,
   'resultTypeID': 2,
   'resultDescription': 'Ergebnis nach Ende der offiziellen Spielzeit'},
  {'resultID': 98426,


In [97]:
home_team = df.iloc[3, 1][0]['homeTeam']['name']
away_team = df.iloc[3, 1][0]['awayTeam']['name']
score_home = df.iloc[3, 1][0]['score']['fullTime']['home']
score_away = df.iloc[3, 1][0]['score']['fullTime']['away']
winner = df.iloc[3, 1][0]['season']['winner']['name']